# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, sunday_to_monday, MO, TH
import plotly_express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)
import sys

In [ ]:
sys.path.append(r"K:\Valuation\_Analysts\Hemanth\Python Notebooks\Miscellaneous\Python Analyst Engine 2.0")

In [ ]:
from util import date_hour_to_peak_block, date_hour_to_time_block

# Inputs

In [ ]:
block_size = 12.5 / 100
wn_start_date = pd.to_datetime('2015-03-01')
wn_end_date = pd.to_datetime('2025-02-28') 

# Defining functions

In [ ]:
def season(row):
    if row['Month'] in [12, 1, 2]:
        return 'Winter'
    elif row['Month'] in [6, 7, 8, 9]:
        return 'Summer'
    else:
        return 'Shoulder'


class NERCHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Year\'s Day', month=1, day=1, observance=sunday_to_monday),
        Holiday('Memorial Day', month=5, day=31, offset=pd.DateOffset(weekday=MO(-1))),
        Holiday('Independence Day', month=7, day=4, observance=sunday_to_monday),
        Holiday('Labor Day', month=9, day=1, offset=pd.DateOffset(weekday=MO(1))),
        Holiday('Thanksgiving', month=11, day=4, offset=pd.DateOffset(weekday=TH(4))),
        Holiday('Christmas Day', month=12, day=25, observance=sunday_to_monday)
    ]

# Importing data and assembling df

In [ ]:
path = r'K:\Valuation\Wholesale for Retail\_Full Requirements\NEPOOL\NSTAR\2025-05\Pri Hemanth\Load\Pri Hemanth NSTAR 2025-05 RES WeatherNorm_v6_48 - Normal weather analysis.xlsb'

df_hourly = pd.read_excel(path, usecols='CZ:DA, DF:DH', skiprows=3, sheet_name='NEMA')

In [ ]:
df_hourly['Excel_Date'] = df_hourly['Date']

In [ ]:
df_hourly['Date'] = pd.to_datetime(df_hourly['Date'], unit='D', origin='1899-12-30')

In [ ]:
df_hourly.dropna(inplace=True)

In [ ]:
# df_hourly

In [ ]:
df_hourly.columns=['Date', 'Hour', 'Load', 'Hub_Price', 'Delivery_Price', 'Excel_Date']

In [ ]:
nerc_cal = NERCHolidayCalendar()

holidays = nerc_cal.holidays(start=df_hourly['Date'].min(), end=df_hourly['Date'].max())

df_hourly = df_hourly.assign(
    Month=lambda DF: DF.Date.dt.month,
    Year=lambda DF: DF.Date.dt.year,
    Season=lambda DF: DF.apply(season, axis=1),
)

df_hourly['Peak_block'] = df_hourly.apply(
    lambda DF: date_hour_to_peak_block(DF['Date'], DF['Hour'], iso='ISONE'), axis=1
)

df_hourly['Time_block'] = df_hourly.apply(
    lambda DF: date_hour_to_peak_block(DF['Date'], DF['Hour'], iso='ISONE'), axis=1
)

df_hourly = df_hourly.assign(
    OFF=lambda DF: np.where(DF.Peak_block != '5x16', 1, 0),
    Weekend=lambda DF: DF.Date.dt.weekday > 4,
    Holiday=lambda DF: DF.Date.isin(holidays)
    
)

df_hourly

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_hourly.Date, y=df_hourly.Load, mode='lines'))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_hourly.Date, y=df_hourly.Delivery_Price - df_hourly.Hub_Price, mode='lines', name='Basis'))

In [ ]:
# px.imshow(df_hourly.pivot_table(index='Month', values='Load', columns='Year', aggfunc='mean') * 0.125)

In [ ]:
df_daily = df_hourly.pivot_table(index='Date', values=['Load', 'Delivery_Price'], aggfunc='mean').sort_index()

In [ ]:
df_daily.reset_index(inplace=True)

df_daily

# Importing temperature data

In [ ]:
# Created additional tab to pull temperature data until 2/28/2025 for calculating "normal" variables

df_daily_temp = pd.read_excel(path, usecols='DP, DX:DZ', skiprows=3, sheet_name='NEMA additional month')

In [ ]:
df_daily_temp

In [ ]:
df_daily_temp.columns = ['Date', 'Avg_temp', 'Max_temp', 'Min_temp']

df_daily_temp.dropna(inplace=True)

df_daily_temp['CDD'] = df_daily_temp.apply(lambda x: max(x['Avg_temp'] - 65, 0), axis=1)
df_daily_temp['HDD'] = df_daily_temp.apply(lambda x: max(65 - x['Avg_temp'], 0), axis=1)

df_daily_temp['CDD^2'] = df_daily_temp['CDD'] ** 2
df_daily_temp['HDD^2'] = df_daily_temp['HDD'] ** 2

df_daily_temp['CDD75'] = df_daily_temp.apply(lambda x: max(x['Max_temp'] - 75, 0), axis=1)
df_daily_temp['HDD40'] = df_daily_temp.apply(lambda x: max(40 - x['Min_temp'], 0), axis=1)

df_daily_temp['CDDLag'] = df_daily_temp['CDD'].shift(1) * 0.75 + df_daily_temp['CDD'].shift(2) * 0.25
df_daily_temp['HDDLag'] = df_daily_temp['HDD'].shift(1) * 0.75 + df_daily_temp['HDD'].shift(2) * 0.25

df_daily_temp['Date'] = pd.to_datetime(df_daily_temp['Date'], unit='D', origin='1899-12-30') # Converting Excel Date to Pandas datetime
df_daily_temp['Day'] = df_daily_temp.Date.dt.day
# df_daily_temp['Month'] = df_daily_temp.Date.dt.month

In [ ]:
# date_filter = (df_daily_temp.Date > wn_start_date) & (df_daily_temp.Date < wn_end_date)

# df_daily_temp = df_daily_temp[date_filter]

df_daily_temp

# Merging with temperature data

In [ ]:
df_daily = df_daily.merge(right=df_daily_temp, how='outer', on='Date', validate='one_to_one')

In [ ]:
df_daily = df_daily.merge(right=df_hourly[['Date', 'Month', 'Season', 'Weekend', 'Holiday', 'Excel_Date']], how='inner', on='Date', validate='one_to_many').drop_duplicates()

# df_daily = df_daily.merge(right=df_hourly[['Date', 'Month', 'Season', 'Excel_Date']], how='inner', on='Date', validate='one_to_many').drop_duplicates()

In [ ]:
df_daily.reset_index(inplace=True, drop=True)

In [ ]:
# Adding additional variables for regression

df_daily['Day'] = df_daily['Date'].dt.day

df_daily['CDD'] = df_daily.apply(lambda x: max(x['Avg_temp'] - 65, 0), axis=1)
df_daily['HDD'] = df_daily.apply(lambda x: max(65 - x['Avg_temp'], 0), axis=1)

df_daily['CDD^2'] = df_daily['CDD'] ** 2
df_daily['HDD^2'] = df_daily['HDD'] ** 2

df_daily['CDD75'] = df_daily.apply(lambda x: max(x['Max_temp'] - 75, 0), axis=1)
df_daily['HDD40'] = df_daily.apply(lambda x: max(40 - x['Min_temp'], 0), axis=1)

df_daily['CDDLag'] = df_daily['CDD'].shift(1) * 0.75 + df_daily['CDD'].shift(2) * 0.25
df_daily['HDDLag'] = df_daily['HDD'].shift(1) * 0.75 + df_daily['HDD'].shift(2) * 0.25

df_daily['Covid'] = df_daily.apply(lambda x: x['Date'] > pd.to_datetime('2020-02-29'), axis=1)
df_daily['Covid_Date'] = df_daily.apply(lambda x: x['Excel_Date'] if(x['Covid'] == 1) else 0, axis=1)

for i in range(12): # indicator variables for months
    col_name = 'Month' + str(i + 1)
    df_daily[col_name] = df_daily.Date.dt.month == (i + 1)

df_daily['OFF'] = df_daily.apply(lambda x: (x['Weekend'] == True) | (x['Holiday'] == True), axis=1)

# df_daily['OFF'] = df_daily.apply(lambda DF: DF['Peak_block'] != '5x16', axis=1)

In [ ]:
df_daily.fillna(method='bfill', inplace=True)

df_daily

# Calculating normal weather variables

In [ ]:
date_filter = (df_daily_temp.Date > wn_start_date) & (df_daily_temp.Date < wn_end_date)

df_daily_temp['Month'] = df_daily_temp.Date.dt.month # adding month for generating pivot tables below

In [ ]:
avg_temp_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='Avg_temp', aggfunc='mean')
min_temp_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='Min_temp', aggfunc='mean')
max_temp_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='Max_temp', aggfunc='mean')

CDD_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='CDD', aggfunc='mean')
HDD_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='HDD', aggfunc='mean')

# This calculates the  CDD^2_normal and HDD^2_normal by averaging the history
# CDD2_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='CDD', aggfunc='mean')
# HDD2_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='HDD', aggfunc='mean')
##########################################################################################

CDD75_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='CDD75', aggfunc='mean')
HDD40_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='HDD40', aggfunc='mean')

CDDLag_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='CDDLag', aggfunc='mean')
HDDLag_normal = df_daily_temp[date_filter].pivot_table(index='Day', columns='Month', values='HDDLag', aggfunc='mean')


In [ ]:
# df_daily[(df_daily.Day == 1) & (df_daily.Month == 1)]['Avg_temp'].mean()

avg_temp_normal

In [ ]:
df_daily['Avg_temp_normal'] = df_daily.apply(lambda x: avg_temp_normal.loc[x['Day'], x['Month']], axis=1)
df_daily['Min_temp_normal'] = df_daily.apply(lambda x: min_temp_normal.loc[x['Day'], x['Month']], axis=1)
df_daily['Max_temp_normal'] = df_daily.apply(lambda x: max_temp_normal.loc[x['Day'], x['Month']], axis=1)

df_daily['CDD_normal'] = df_daily.apply(lambda x: CDD_normal.loc[x['Day'], x['Month']], axis=1)
df_daily['HDD_normal'] = df_daily.apply(lambda x: HDD_normal.loc[x['Day'], x['Month']], axis=1)

df_daily['CDD75_normal'] = df_daily.apply(lambda x: CDD75_normal.loc[x['Day'], x['Month']], axis=1)
df_daily['HDD40_normal'] = df_daily.apply(lambda x: HDD40_normal.loc[x['Day'], x['Month']], axis=1)

# This can be used if we want the CDD^2_normal and HDD^2_normal by averaging history
# df_daily['CDD^2_normal'] = df_daily.apply(lambda x: CDD2_normal.loc[x['Day'], x['Month']], axis=1)
# df_daily['HDD^2_normal'] = df_daily.apply(lambda x: HDD2_normal.loc[x['Day'], x['Month']], axis=1)

# This squares CDD_normal and HDD_normal to get the CDD^2_normal and HDD^2_normal
df_daily['CDD^2_normal'] = df_daily['CDD_normal'] ** 2
df_daily['HDD^2_normal'] = df_daily['HDD_normal'] ** 2

df_daily['CDDLag_normal'] = df_daily.apply(lambda x: CDDLag_normal.loc[x['Day'], x['Month']], axis=1)
df_daily['HDDLag_normal'] = df_daily.apply(lambda x: HDDLag_normal.loc[x['Day'], x['Month']], axis=1)

df_daily

In [ ]:
# Just doing an aggregated version of the normal weather variables to come up with an equation of load as a function of price 

df_daily[
    [
        'Date',
        'Avg_temp_normal',
        'Min_temp_normal',
        'Max_temp_normal',
        'CDD_normal',
        'HDD_normal',
        'CDD75_normal',
        'HDD40_normal',
        'CDD^2_normal',
        'HDD^2_normal',
        'CDDLag_normal',
        'HDDLag_normal'

    ]
].assign(
    Month=lambda DF: DF.Date.dt.month
).drop(
    columns='Date'
).groupby('Month').mean().loc[6] # For June as an example

In [ ]:
# df_daily[(df_daily.Month == 1) & (df_daily.Day == 1)]['Avg_temp'][0:11].sum() / 10

In [ ]:
# Setting these columns to 0 to ensure matrix is invertible

df_daily_summer = df_daily[df_daily.Season == 'Summer']
df_daily_summer.loc[:, 'Month8'] = False

df_daily_winter = df_daily[df_daily.Season == 'Winter']
df_daily_winter.loc[:, 'Month2'] = False

df_daily_shoulder = df_daily[df_daily.Season == 'Shoulder']
df_daily_shoulder.loc[:, 'Month3'] = False

# Summer Regression

In [ ]:
# For training the regression model - as an experiment, also adding the Delivery_Price
regression_vars_summer = ['Excel_Date', 'OFF', 'CDD', 'CDD^2', 'CDDLag', 'CDD75', 'Month6', 'Month7', 'Month8', 'Month9', 'Covid', 'Covid_Date', 'Delivery_Price']

# For predicting the "normal" load
regression_vars_summer_normal = ['Excel_Date', 'OFF', 'CDD_normal', 'CDD^2_normal', 'CDDLag_normal', 'CDD75_normal', 'Month6', 'Month7', 'Month8', 'Month9', 'Covid', 'Covid_Date','Delivery_Price']

# Fitting the regression model
regressor_summer = LinearRegression().fit(df_daily_summer[regression_vars_summer], df_daily_summer.Load.values.reshape(-1, 1))

In [ ]:
# Predicting summer load (what the WN calls forecasted load/regression load)
y_pred_summer = regressor_summer.predict(df_daily_summer[regression_vars_summer])

In [ ]:
# Calculating residuals for the summer
residuals_summer = df_daily_summer.Load - y_pred_summer.reshape(-1, )

In [ ]:
# Renaming columns so that same df can be used for predicting the "normal" load
df_daily_summer_normal = df_daily_summer[regression_vars_summer_normal].rename(columns=dict(zip(regression_vars_summer_normal, regression_vars_summer)))

# Incorporating the residuals into the predicted "normal" load
y_pred_summer_normal = regressor_summer.predict(df_daily_summer_normal).reshape(-1, ) + residuals_summer

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_daily_summer.Avg_temp, y=df_daily_summer.Load, mode='markers', name='Summer load'))
fig.add_trace(go.Scatter(x=df_daily_summer.Avg_temp.values, y=y_pred_summer.reshape(-1, ), mode='markers', name='Regression load'))
fig.add_trace(go.Scatter(x=df_daily_summer.Avg_temp.values, y=y_pred_summer_normal, mode='markers', name='Normal load')) 
# Had to reshape the regression output to make the plot work

fig.show()

In [ ]:
# Printing out regression coefficients, intercept and R2

for i in range(len(regression_vars_summer)):
    print(regression_vars_summer[i], round(regressor_summer.coef_[0, i], 3))

print('Intercept', round(regressor_summer.intercept_[0], 0))

print('R2 score:', round(r2_score(y_pred_summer, df_daily_summer.Load), 4))

In [ ]:
(regressor_summer.coef_ * np.array([[0, 0, 3.6477, 16.28, 3.41, 3.29, 1, 0, 0, 0, 1, 0, 0]])).sum() + 199

# Winter Regression

In [ ]:
# For training the regression model - as an experiment, also adding the Delivery_Price
regression_vars_winter = ['Excel_Date', 'OFF', 'HDD', 'HDD^2', 'HDDLag', 'HDD40', 'Month1', 'Month2', 'Month12', 'Covid', 'Covid_Date', 'Delivery_Price']

# For predicting the "normal" load - as an experiment, also adding the Delivery_Price
regression_vars_winter_normal = ['Excel_Date', 'OFF', 'HDD_normal', 'HDD^2_normal', 'HDDLag_normal', 'HDD40_normal', 'Month1', 'Month2', 'Month12', 'Covid', 'Covid_Date', 'Delivery_Price']

# Fitting the regression model
regressor_winter = LinearRegression().fit(df_daily_winter[regression_vars_winter], df_daily_winter.Load.values.reshape(-1, 1))

In [ ]:
# Predicting winter load (what the WN calls forecasted load/regression load)
y_pred_winter = regressor_winter.predict(df_daily_winter[regression_vars_winter])

In [ ]:
# Calculating residuals for the winter
residuals_winter = df_daily_winter.Load - y_pred_winter.reshape(-1, )

In [ ]:
# Renaming columns so that same df can be used for predicting the "normal" load
df_daily_winter_normal = df_daily_winter[regression_vars_winter_normal].rename(columns=dict(zip(regression_vars_winter_normal, regression_vars_winter)))

# Incorporating the residuals into the predicted "normal" load
y_pred_winter_normal = regressor_winter.predict(df_daily_winter_normal).reshape(-1, ) + residuals_winter

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_daily_winter.Avg_temp, y=df_daily_winter.Load, mode='markers', name='Winter load'))
fig.add_trace(go.Scatter(x=df_daily_winter.Avg_temp.values, y=y_pred_winter.reshape(-1, ), mode='markers', name='Regression')) 
fig.add_trace(go.Scatter(x=df_daily_winter.Avg_temp.values, y=y_pred_winter_normal, mode='markers', name='Normal load'))
# Had to reshape the regression output to make the plot work

fig.show()

In [ ]:
for i in range(len(regression_vars_winter)):
    print(regression_vars_winter[i], round(regressor_winter.coef_[0, i], 3))

print('Intercept', round(regressor_winter.intercept_[0], 0))

print('R2 score:', round(r2_score(y_pred_winter.reshape(-1, ), df_daily_winter.Load), 3))

# Shoulder Regression

In [ ]:
# For training the regression model - as an experiment, also adding the Delivery_Price
regression_vars_shoulder = ['Excel_Date', 'OFF', 'CDD', 'CDD^2', 'CDDLag', 'HDD', 'HDD^2', 'HDDLag', 'HDD40', 'Month3', 'Month4', 'Month5', 'Month10', 'Month11', 'Covid', 'Covid_Date', 'Delivery_Price']

# For predicting the "normal" load - as an experiment, also adding the Delivery_Price
regression_vars_shoulder_normal = ['Excel_Date', 'OFF', 'CDD_normal', 'CDD^2_normal', 'CDDLag_normal', 'HDD_normal', 'HDD^2_normal', 'HDDLag_normal', 'HDD40_normal', 'Month3', 'Month4', 'Month5', 'Month10', 'Month11', 'Covid', 'Covid_Date', 'Delivery_Price']

# Fitting the regression model
regressor_shoulder = LinearRegression().fit(df_daily_shoulder[regression_vars_shoulder], df_daily_shoulder.Load.values.reshape(-1, 1))

In [ ]:
# Predicting shoulder load (what the WN calls forecasted load/regression load)
y_pred_shoulder = regressor_shoulder.predict(df_daily_shoulder[regression_vars_shoulder])

In [ ]:
# Calculating residuals for the shoulder
residuals_shoulder = df_daily_shoulder.Load - y_pred_shoulder.reshape(-1, )

In [ ]:
# Renaming columns so that same df can be used for predicting the "normal" load
df_daily_shoulder_normal = df_daily_shoulder[regression_vars_shoulder_normal].rename(columns=dict(zip(regression_vars_shoulder_normal, regression_vars_shoulder)))

# Incorporating the residuals into the predicted "normal" load
y_pred_shoulder_normal = regressor_shoulder.predict(df_daily_shoulder_normal).reshape(-1, ) + residuals_shoulder

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_daily_shoulder.Avg_temp, y=df_daily_shoulder.Load, mode='markers', name='Shoulder load'))
fig.add_trace(go.Scatter(x=df_daily_shoulder.Avg_temp.values, y=y_pred_shoulder.reshape(-1, ), mode='markers', name='Regression'))
fig.add_trace(go.Scatter(x=df_daily_shoulder.Avg_temp.values, y=y_pred_shoulder_normal, mode='markers', name='Normal load')) 
# Had to reshape the regression output to make the plot work

fig.show()

In [ ]:
for i in range(len(regression_vars_shoulder)):
    print(regression_vars_shoulder[i], round(regressor_shoulder.coef_[0, i], 3))

print('Intercept', round(regressor_shoulder.intercept_[0], 3))

print('R2 score:', round(r2_score(y_pred_shoulder.reshape(-1, ), df_daily_shoulder.Load), 3))

In [ ]:
# For summer, winter and shoulder - I know why my R^2 doesn't exactly tie to the WN model even though the coefficients are very close. 
# It is because of the difference in normal weather calculation - difference in the range of temperatures used to calculate normal weather

# Combined plots

In [ ]:
fig = go.Figure()

# Had to reshape the regression output to make the plot work

fig.add_trace(
    go.Scatter(
        x=pd.concat([df_daily_summer.Avg_temp, df_daily_winter.Avg_temp, df_daily_shoulder.Avg_temp]),
        y=pd.concat([df_daily_summer.Load, df_daily_winter.Load, df_daily_shoulder.Load]),
        mode='markers',
        name='Load'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.concatenate((df_daily_summer.Avg_temp.values, df_daily_winter.Avg_temp.values, df_daily_shoulder.Avg_temp.values)),
        y=np.concatenate((y_pred_summer, y_pred_winter, y_pred_shoulder)).reshape(-1, ),
        mode='markers',
        name='Regression'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.concatenate((df_daily_summer.Avg_temp.values, df_daily_winter.Avg_temp.values, df_daily_shoulder.Avg_temp.values)),
        y=np.concatenate((y_pred_summer_normal, y_pred_winter_normal, y_pred_shoulder_normal)).reshape(-1, ),
        mode='markers',
        name='Normal'
    )
)

fig.update_layout(title='Raw load, Regressed load and Normal load vs Temperature')
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=pd.concat([df_daily_summer.Date, df_daily_winter.Date, df_daily_shoulder.Date]), 
        y=np.concatenate((residuals_summer, residuals_winter, residuals_shoulder)), 
        mode='markers'
    )
)

fig.update_layout(title='Residuals vs Time')
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.concatenate((df_daily_summer.Avg_temp.values, df_daily_winter.Avg_temp.values, df_daily_shoulder.Avg_temp.values)), 
        y=np.concatenate((residuals_summer, residuals_winter, residuals_shoulder)),
        mode='markers', 
    )
)

fig.update_layout(title='Residuals vs Avg Temp')
fig.show()

# Generating WN output tables

## Incorporating regression results into df_daily

In [ ]:
df_WN_results = pd.DataFrame(
    {
        'Date': pd.concat([df_daily_summer.Date, df_daily_winter.Date, df_daily_shoulder.Date]),
        'WN_load_with_residuals': np.concatenate((y_pred_summer_normal, y_pred_winter_normal, y_pred_shoulder_normal)).reshape(-1, ),
        'Predicted_load': np.concatenate((y_pred_summer, y_pred_winter, y_pred_shoulder)).reshape(-1, ), # or regression load
        'Residuals': np.concatenate((residuals_summer, residuals_winter, residuals_shoulder)).reshape(-1, ),
        }
        )

df_WN_results

In [ ]:
# Merging df_daily with WN results

df_daily = df_daily.merge(df_WN_results, how='outer', on='Date', validate='1:1')

In [ ]:
df_daily['Adjustments'] = df_daily['WN_load_with_residuals'] - df_daily['Load']
df_daily['Year'] = df_daily.Date.dt.year # including year for creating summary pivot tables
df_daily['WN_load_without_residuals'] = df_daily['WN_load_with_residuals'] - df_daily['Residuals']

# df_daily[df_daily.Date == pd.to_datetime('2014-06-01')]

df_daily

## WN load with residuals

In [ ]:
block_size * df_daily.pivot_table(index='Month', columns='Year', values='WN_load_with_residuals')

## Predicted/Regressed load

In [ ]:
(block_size * df_daily.pivot_table(index='Month', columns='Year', values='Predicted_load'))

## Residuals

In [ ]:
(block_size * df_daily.pivot_table(index='Month', columns='Year', values='Residuals'))

## Residuals as %

In [ ]:
(df_daily.pivot_table(index='Month', columns='Year', values='Residuals') / df_daily.pivot_table(index='Month', columns='Year', values='Predicted_load')) * 100

## Historical load

In [ ]:
df_daily.pivot_table(index='Month', columns='Year', values='Load') * block_size

## Adjustments as %

In [ ]:
(df_daily.pivot_table(index='Month', columns='Year', values='Adjustments') / df_daily.pivot_table(index='Month', columns='Year', values='Load')) * 100

## WN load without residuals

In [ ]:
(block_size * df_daily.pivot_table(index='Month', columns='Year', values='WN_load_without_residuals'))

# Load Flex Charts

In [ ]:
df_daily['Load_flex'] = (df_daily['WN_load_with_residuals'] - df_daily['Load']) / df_daily['Load']

df_daily

In [ ]:
df_daily.pivot_table(index='Month', columns='Year', values='Load_flex')

In [ ]:
px.imshow(df_daily.pivot_table(index='Month', columns='Year', values='Load_flex'))